In [1]:
import tkinter as tk
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# ตัวอย่างข้อมูล province_id และ province_name
provinces = [
    (1, "กรุงเทพมหานคร"),
    (2, "จังหวัดกาญจนบุรี"),
    (3, "จังหวัดบุรีรัมย์"),
    (4, "จังหวัดขอนแก่น"),
    (5, "จังหวัดจันทบุรี"),
    (6, "จังหวัดชลบุรี"),
    (7, "จังหวัดเชียงใหม่"),
    (8, "จังหวัดเชียงราย"),
    (9, "จังหวัดเชียงใหม่"),
    (10, "จังหวัดตรัง")
]


model = load_model('model.h5')

# คอมไพล์โมเดล
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# โหลด scaler (ถ้าคุณมีการเซฟ scaler ในไฟล์อื่นๆ ใช้ pickle หรือ joblib)
scaler = MinMaxScaler()

# ตัวอย่าง DataFrame แทน X ที่ถูกต้อง
X = pd.DataFrame({
    'Initial_Year': [2023, 2023],
    'Initial_Month': [12, 12],
    'Forecast_Year': [2023, 2023],
    'Forecast_Month': [12, 12],
    'Province_ID': [1, 2]
})

# ฟังก์ชันการพยากรณ์ฝน
def predict_rain(model, scaler, input_data):
    # ตรวจสอบความยาวของข้อมูล
    if len(input_data) == 4:
        last_weather_data = X.iloc[-1][['Initial_Year', 'Initial_Month', 'Forecast_Year', 'Forecast_Month']].values
        input_data = np.concatenate([last_weather_data, input_data])
    
    # Normalize ข้อมูล
    input_data_scaled = scaler.transform([input_data])

    # พยากรณ์ด้วยโมเดล
    predict = model.predict(input_data_scaled)
    probability = predict[0][0] * 100  # Convert to percentage
    
    # Return ผลลัพธ์
    if probability > 50:
        return probability, "Rain is likely"
    else:
        return probability, "No Rain expected"

# ฟังก์ชันสำหรับคำนวณผลลัพธ์
def calculate():
    try:
        iD = int(day_entry.get())
        iM = int(month_entry.get())
        iY = int(year_entry.get())
        city_id = int(city_id_entry.get())
        
        # ตรวจสอบความถูกต้องของข้อมูล
        if not (1 <= iD <= 31):
            result_label.config(text="วันที่ไม่ถูกต้อง (ควรอยู่ระหว่าง 1-31)")
        elif not (1 <= iM <= 12):
            result_label.config(text="เดือนที่ไม่ถูกต้อง (ควรอยู่ระหว่าง 1-12)")
        elif not (city_id in [id for id, _ in provinces]):
            result_label.config(text="ID เมืองไม่ถูกต้อง (กรุณาเลือก ID เมืองที่ถูกต้อง)")
        else:
            # ดำเนินการพยากรณ์
            result_custom_date = predict_rain(model, scaler, np.array([iD, iM, iY, city_id]))
            result_label.config(text=f"ผลลัพธ์: {result_custom_date[1]} (ความน่าจะเป็น: {result_custom_date[0]:.2f}%)")
    
    except ValueError:
        result_label.config(text="กรุณากรอกข้อมูลให้ถูกต้อง")

# สร้าง UI ด้วย tkinter
root = tk.Tk()
root.title("Rain Prediction")

# Labels and Entries
tk.Label(root, text="Day:").grid(row=0, column=0)
day_entry = tk.Entry(root)
day_entry.grid(row=0, column=1)

tk.Label(root, text="Month:").grid(row=1, column=0)
month_entry = tk.Entry(root)
month_entry.grid(row=1, column=1)

tk.Label(root, text="Year:").grid(row=2, column=0)
year_entry = tk.Entry(root)
year_entry.grid(row=2, column=1)

tk.Label(root, text="City ID:").grid(row=3, column=0)
city_id_entry = tk.Entry(root)
city_id_entry.grid(row=3, column=1)

# ปุ่มกดคำนวณ
calculate_button = tk.Button(root, text="คำนวณ", command=calculate)
calculate_button.grid(row=4, column=0, columnspan=2)

# Label สำหรับแสดงผลลัพธ์
result_label = tk.Label(root, text="")
result_label.grid(row=5, column=0, columnspan=2)

root.mainloop()
